## Dependencies

In [1]:
import glob
import warnings
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate, Add
from googleqa_utilityscript import *
from transformers import BertConfig, BertTokenizer, TFBertModel


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/bert-base-uncased-huggingface-transformer/bert-base-uncased-tf_model.h5'
VOCAB_PATH = '/kaggle/input/bert-base-uncased-huggingface-transformer/bert-base-uncased-vocab.txt'
model_path_list = glob.glob('/kaggle/input/144-googleq-a-train-3fold-bert-base-unc-sum/' + '*1.h5')
model_path_list += glob.glob('/kaggle/input/144-googleq-a-train-3fold-bert-base-unc-sum/' + '*2.h5')
model_path_list += glob.glob('/kaggle/input/144-googleq-a-train-3fold-bert-base-unc-sum/' + '*3.h5')
model_path_list.sort()
print('Models to predict:')
print(*model_path_list, sep = "\n")

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict:
/kaggle/input/144-googleq-a-train-3fold-bert-base-unc-sum/model_fold_1.h5
/kaggle/input/144-googleq-a-train-3fold-bert-base-unc-sum/model_fold_2.h5
/kaggle/input/144-googleq-a-train-3fold-bert-base-unc-sum/model_fold_3.h5
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = BertTokenizer.from_pretrained(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
bert_config = BertConfig()
bert_config.output_hidden_states=True

def model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_model = TFBertModel.from_pretrained(BERT_PATH, config=bert_config)
    sequence_output, pooled_output, hidden_states = bert_model([input_word_ids, input_masks, segment_ids])
    
    h12 = tf.reshape(hidden_states[-1][:,0],(-1,1,768))
    h11 = tf.reshape(hidden_states[-2][:,0],(-1,1,768))
    h10 = tf.reshape(hidden_states[-3][:,0],(-1,1,768))
    h09 = tf.reshape(hidden_states[-4][:,0],(-1,1,768))
    
    x = Add()([h12,h11, h10, h09])

    x = GlobalAveragePooling1D()(x)
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS, activation="sigmoid", name="output")(x)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.948285,0.629373,0.307170,0.520458,0.445100,0.451633,0.686436,0.657275,0.564782,...,0.920173,0.917082,0.593788,0.967245,0.958191,0.815580,0.054319,0.051296,0.833725,0.918576
1,46,0.881548,0.530951,0.007443,0.748774,0.767684,0.921689,0.554208,0.493542,0.082854,...,0.743798,0.952569,0.668342,0.972400,0.979684,0.884909,0.924354,0.109564,0.091159,0.891053
2,70,0.926713,0.628690,0.046587,0.717968,0.849286,0.910099,0.612780,0.506378,0.178614,...,0.880493,0.946907,0.637801,0.974749,0.973344,0.856865,0.128585,0.069206,0.834415,0.915795
3,132,0.898983,0.472293,0.010451,0.684528,0.707618,0.894894,0.561879,0.479629,0.067670,...,0.739057,0.953963,0.673750,0.974830,0.981037,0.893526,0.876494,0.148333,0.352332,0.901842
4,200,0.954668,0.456196,0.053312,0.844847,0.648977,0.793689,0.659477,0.584116,0.070950,...,0.739745,0.924212,0.629061,0.974047,0.976939,0.859989,0.250874,0.084722,0.591522,0.909266


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.904801,0.610495,0.049128,0.703334,0.791217,0.843375,0.591726,0.519872,0.218704,...,0.823889,0.944920,0.669054,0.970656,0.974361,0.876449,0.539824,0.121881,0.462861,0.911077
std,2812.670060,0.037471,0.126223,0.091946,0.104036,0.105881,0.130301,0.048909,0.079256,0.188849,...,0.073455,0.016219,0.038648,0.006770,0.008195,0.033885,0.327779,0.054156,0.276880,0.015992
min,39.000000,0.770502,0.368572,0.003682,0.230290,0.287303,0.256271,0.499289,0.385490,0.008888,...,0.666584,0.886403,0.565371,0.939527,0.940738,0.752666,0.004010,0.006229,0.025539,0.838642
25%,2572.000000,0.880755,0.495996,0.009333,0.647541,0.742909,0.824966,0.552234,0.464003,0.070813,...,0.763810,0.936334,0.644025,0.966954,0.969736,0.856721,0.200383,0.082007,0.221687,0.901443
50%,5093.000000,0.906521,0.608160,0.015090,0.706250,0.800454,0.895175,0.582348,0.500056,0.147697,...,0.824716,0.948181,0.670545,0.971678,0.975873,0.881921,0.647296,0.122835,0.434127,0.912691
75%,7482.000000,0.935484,0.717272,0.037294,0.764708,0.858424,0.921266,0.624932,0.556102,0.320245,...,0.890058,0.956846,0.695312,0.975623,0.980433,0.899943,0.834453,0.156825,0.678618,0.921433
max,9640.000000,0.980485,0.886020,0.691109,0.960493,0.974990,0.965564,0.747343,0.795776,0.875048,...,0.957146,0.977123,0.779929,0.984969,0.989376,0.964563,0.959144,0.250089,0.986599,0.953813
